# RLLM SDK Quick Start: Make Any Agent Trainable

This tutorial shows how to make **any existing agent code** trainable with minimal changes.

**The key insight:** Just replace your OpenAI client with the SDK client, and everything is automatically tracked for training!

## Prerequisites

```bash
pip install -e .
pip install litellm openai
export OPENAI_API_KEY="sk-..."
```

## Step 1: Start the Proxy

One-time setup - start the proxy manager that tracks all LLM calls:

In [ ]:
import os
from pathlib import Path
from rllm.sdk.proxy.proxy_manager import ProxyManager

# Setup
DB_PATH = "/tmp/rllm_demo.db"
MODEL = "gpt-4o-mini"

# Clean up
Path(DB_PATH).unlink(missing_ok=True)

# Start proxy
proxy_manager = ProxyManager(proxy_port=4000)
config = {
    "model_list": [{
        "model_name": MODEL,
        "litellm_params": {
            "model": MODEL,
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    }]
}
proxy_manager.start_proxy_subprocess(config=config, db_path=DB_PATH, project="demo")
proxy_url = proxy_manager.get_proxy_url(include_v1=True)

print(f"✓ Proxy started at {proxy_url}")
print(f"✓ Database: {DB_PATH}")

## Step 2: Your Original Agent Code

Here's a typical agent using the standard OpenAI client:

In [ ]:
from openai import AsyncOpenAI

class MathSolver:
    """A simple math solving agent - ORIGINAL VERSION"""
    
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # Standard OpenAI client
        self.client = AsyncOpenAI(api_key=api_key)
        self.model = model
    
    async def solve(self, problem: str) -> str:
        """Solve a math problem."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "user", "content": f"Solve this problem: {problem}"}
            ],
            max_tokens=200,
        )
        return response.choices[0].message.content

# Use it
original_solver = MathSolver(api_key=os.environ["OPENAI_API_KEY"])
result = await original_solver.solve("What is 25 + 17?")

print("Original Agent Result:")
print(result)
print("\n❌ This call is NOT tracked - can't use for training!")

## Step 3: Make It Trainable (2 Simple Changes!)

**Change 1:** Import the SDK client instead of OpenAI client  
**Change 2:** Point to the proxy URL

That's it! Everything else stays the same:

In [ ]:
from rllm.sdk import get_chat_client_async, session  # Change 1: Import SDK client

class TrainableMathSolver:
    """A simple math solving agent - SDK VERSION (trainable!)"""
    
    def __init__(self, base_url: str, model: str = "gpt-4o-mini"):
        # Change 2: Use SDK client pointing to proxy
        self.client = get_chat_client_async(base_url=base_url, api_key="EMPTY")
        self.model = model
    
    async def solve(self, problem: str) -> str:
        """Solve a math problem - now tracked!"""
        # Wrap in session context to track this rollout
        with session(task="math_solving") as sess:
            response = await self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": f"Solve this problem: {problem}"}
                ],
                max_tokens=200,
            )
            # Store session UID for later retrieval
            self.last_session_uid = sess._uid
            return response.choices[0].message.content

# Use it - exact same interface!
trainable_solver = TrainableMathSolver(base_url=proxy_url)
result = await trainable_solver.solve("What is 25 + 17?")

print("SDK Agent Result:")
print(result)
print(f"\n✅ This call IS tracked! Session: {trainable_solver.last_session_uid}")

## Step 4: Retrieve the Traces

Now you can retrieve **everything** about that interaction:

In [ ]:
from rllm.sdk.store import SqliteTraceStore

# Flush to ensure traces are written
await proxy_manager.flush_tracer(timeout=10.0)

# Retrieve traces by session UID
store = SqliteTraceStore(db_path=DB_PATH)
traces = await store.get_by_session_uid(trainable_solver.last_session_uid)

print(f"✅ Retrieved {len(traces)} trace(s)\n")

# Inspect the trace
trace = traces[0]
data = trace.data

print("=" * 70)
print("TRACE DETAILS")
print("=" * 70)
print(f"Model: {data['model']}")
print(f"Latency: {data['latency_ms']:.2f} ms")
print(f"Tokens: {data['tokens']['total']} (prompt: {data['tokens']['prompt']}, completion: {data['tokens']['completion']})")
print(f"\nInput Messages:")
for msg in data['input']['messages']:
    print(f"  [{msg['role']}]: {msg['content']}")
print(f"\nOutput:")
print(f"  {data['output']['choices'][0]['message']['content']}")
print("=" * 70)

print("\n💡 This trace contains everything you need for training!")

## Step 5: Add Rewards and Train

Now you can add rewards and use this for training:

In [ ]:
# Example: Run multiple problems and collect traces
problems = [
    {"question": "What is 15 + 27?", "answer": "42"},
    {"question": "What is 8 * 7?", "answer": "56"},
    {"question": "What is 100 - 37?", "answer": "63"},
]

print("Running rollouts...\n")

session_uids = []
for i, problem in enumerate(problems, 1):
    result = await trainable_solver.solve(problem["question"])
    session_uids.append(trainable_solver.last_session_uid)
    
    # Simple reward: check if answer is in response
    reward = 1.0 if problem["answer"] in result else 0.0
    
    print(f"Problem {i}: {problem['question']}")
    print(f"  Response: {result[:100]}..." if len(result) > 100 else f"  Response: {result}")
    print(f"  Reward: {'✓' if reward == 1.0 else '✗'} {reward}")
    print()

print(f"✅ Collected {len(session_uids)} rollouts")
print(f"✅ All traces stored in {DB_PATH}")
print(f"\n💡 You can now use these traces for:")
print("   - Supervised Fine-Tuning (SFT)")
print("   - Reinforcement Learning (PPO)")
print("   - Debugging and analysis")

## Bonus: Using @trajectory Decorator (Even Easier!)

For even cleaner code, use the `@trajectory` decorator:

In [ ]:
from rllm.sdk import trajectory

class DecoratedMathSolver:
    """Math solver using @trajectory decorator - cleanest version!"""
    
    def __init__(self, base_url: str, model: str = "gpt-4o-mini"):
        self.client = get_chat_client_async(base_url=base_url, api_key="EMPTY")
        self.model = model
    
    @trajectory(name="solver")  # Automatically creates session and tracks!
    async def solve(self, problem: str) -> str:
        """Solve a math problem."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "user", "content": f"Solve this problem: {problem}"}
            ],
            max_tokens=200,
        )
        return response.choices[0].message.content

# Use it
decorated_solver = DecoratedMathSolver(base_url=proxy_url)
traj = await decorated_solver.solve("What is 12 * 8?")

print("Decorator Version:")
print(f"  Result: {traj.result}")
print(f"  Trajectory Name: {traj.name}")
print(f"  Number of Steps: {len(traj.steps)}")
print(f"  Step 0 Trace ID: {traj.steps[0].id}")
print("\n💡 The @trajectory decorator automatically returns a TrajectoryView!")

## Summary: From Code to Training in 3 Steps

### 1. Start Proxy (once)
```python
proxy_manager = ProxyManager(proxy_port=4000)
proxy_manager.start_proxy_subprocess(config=config, db_path="traces.db")
```

### 2. Replace Client (2 lines)
```python
# Before:
from openai import AsyncOpenAI
client = AsyncOpenAI(api_key=api_key)

# After:
from rllm.sdk import get_chat_client_async, session
client = get_chat_client_async(base_url=proxy_url, api_key="EMPTY")
```

### 3. Wrap in Session
```python
with session(task="my_task") as sess:
    response = await client.chat.completions.create(...)  # Automatically tracked!
```

### Retrieve Traces
```python
# By session UID
traces = await store.get_by_session_uid(sess._uid)
```

**That's it!** All your agent interactions are now:
- ✅ Fully traced (inputs, outputs, tokens, latency)
- ✅ Stored in SQLite
- ✅ Ready for training (SFT, PPO)
- ✅ Retrievable for debugging and analysis

## Next Steps

- **Training:** See `tutorial.ipynb` for full SFT and PPO training examples
- **Multi-Agent:** Check `examples/sdk/solver_judge/` for complex workflows
- **Production:** See `examples/sdk/simple_math/` for complete training scripts

In [ ]:
# Cleanup
proxy_manager.shutdown_proxy()
print("✓ Proxy shutdown complete")